In [0]:
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [2]:
# Enter the ID of the Google Drive folder containing train.csv and test.csv files.
# The ID of the folder is the long string of numbers and letters in the URL of the folder in Google Drive.

file_list = drive.ListFile({'q': "'1V76PUZB9NA-kWRJmJdDGbtnCidNWpT9C' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

title: HousingPrices.ipynb, id: 1Bi7AmvHdffrqgVLjBNCdHtSbPYzT44Wp
title: train (1), id: 1_2k3JMa33JeuO29WjNq8Tvz15ySqG1sVUUQmEQCM6yg
title: test (1).csv, id: 12g-Ro232gNbmTHKW9z3_fFDKrLpEJRTQ
title: train (1).csv, id: 1ygU2lQVVRTE1x_kMXD7MCNb1q7irc2ts


In [0]:
# Enter the ID of train.csv and test.csv. The IDs are printed in the output of the cell above.
train_downloaded = drive.CreateFile({'id': '1ygU2lQVVRTE1x_kMXD7MCNb1q7irc2ts'})
train_downloaded.GetContentFile('train.csv')
test_downloaded = drive.CreateFile({'id': '12g-Ro232gNbmTHKW9z3_fFDKrLpEJRTQ'})
test_downloaded.GetContentFile('test.csv')

In [4]:
# See the top rows of the train.csv file
!ls
!head train.csv
!head test.csv
#!wc -l train.csv


adc.json  sample_data  test.csv  train.csv
Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
1,60,RL,65,8450,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,Co

In [0]:
# Import pandas, which is the library for the data structures being used
import tensorflow as tf
import pandas as pd
from dateutil import parser
import datetime
from datetime import date
import math

# Load train.csv into pandas dataframe and print the summary
df = pd.read_csv('train.csv')
df1 = pd.read_csv('test.csv')


In [6]:
# Show shapes of the data
print ("Train data shape:", df.shape)
print ("Test data shape:", df1.shape)
print list(df)
print list(df1)

import tensorflow as tf
import pandas as pd
from dateutil import parser
import datetime
from datetime import date
import math



def get_features(row):
  complete = True
  SalePrice = row['SalePrice']
  OverallQual = row['OverallQual']
  OverallCond = row['OverallCond']
  YearBuilt = row['YearBuilt']
  FullBath = row['FullBath']
  BedroomAbvGr = row['BedroomAbvGr']
  HalfBath = row['HalfBath']
  LotArea = row['LotArea']
  GrLivArea = row['GrLivArea']
  Neighborhood = row['Neighborhood']
  if math.isnan(SalePrice) \
    or math.isnan(OverallQual) \
    or math.isnan(OverallCond) \
    or math.isnan(YearBuilt) \
    or math.isnan(FullBath) \
    or math.isnan(BedroomAbvGr):
    complete = False
  features = {
      "SalePrice": tf.train.Feature(float_list = tf.train.FloatList(value=[math.log(SalePrice)])),
      "OverallQual": tf.train.Feature(float_list = tf.train.FloatList(value=[OverallQual])),
      "OverallCond": tf.train.Feature(float_list = tf.train.FloatList(value=[OverallCond])),
      "YearBuilt": tf.train.Feature(float_list = tf.train.FloatList(value=[(YearBuilt - 1990.) / 10.])),
      "FullBath": tf.train.Feature(float_list = tf.train.FloatList(value=[FullBath])),
      "BedroomAbvGr": tf.train.Feature(float_list = tf.train.FloatList(value=[BedroomAbvGr])),
      "HalfBath": tf.train.Feature(float_list = tf.train.FloatList(value=[HalfBath])),
      "LotArea": tf.train.Feature(float_list = tf.train.FloatList(value=[LotArea / 1000.0])),
      "GrLivArea": tf.train.Feature(float_list = tf.train.FloatList(value=[GrLivArea / 1000.0])),
      "Neighborhood": tf.train.Feature(bytes_list = tf.train.BytesList(value=[Neighborhood.encode("utf8")])),
  }
  return (features, complete)

reader = pd.read_csv('train.csv', iterator=True)
counter = 0
for j in range(14):
  tfr_file = ("training_examples%d.tfr" % j)
  print ("generating TFRecord file " + tfr_file)
  writer = tf.python_io.TFRecordWriter(tfr_file)

  # For int features:
  # 'name': tf.train.Feature(int64_list = tf.train.Int64List(value = ...))
  # For float features:
  # 'name': tf.train.Feature(float_list = tf.train.FloatList(value = ...))

  for i in range(10):
    print ("processing chunk #" + str(i))
    df = reader.get_chunk(10)

    for index, row in df.iterrows():
      feature = get_features(row)
      if feature[1]:
        counter=counter+1
        example = tf.train.Example(
            features=tf.train.Features(feature=feature[0]))
        writer.write(example.SerializeToString())
        #print example

  writer.close()
print(counter)


('Train data shape:', (1460, 81))
('Test data shape:', (1459, 80))
['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF'

In [7]:
!ls -l

total 1296
-rw-r--r-- 1 root root   2494 Apr  2 20:45 adc.json
drwxr-xr-x 1 root root   4096 Mar 27 20:26 sample_data
-rw-r--r-- 1 root root 451405 Apr  2 20:45 test.csv
-rw-r--r-- 1 root root 460676 Apr  2 20:45 train.csv
-rw-r--r-- 1 root root  25744 Apr  2 20:45 training_examples0.tfr
-rw-r--r-- 1 root root  25747 Apr  2 20:45 training_examples10.tfr
-rw-r--r-- 1 root root  25754 Apr  2 20:45 training_examples11.tfr
-rw-r--r-- 1 root root  25730 Apr  2 20:45 training_examples12.tfr
-rw-r--r-- 1 root root  25752 Apr  2 20:45 training_examples13.tfr
-rw-r--r-- 1 root root  25757 Apr  2 20:45 training_examples1.tfr
-rw-r--r-- 1 root root  25755 Apr  2 20:45 training_examples2.tfr
-rw-r--r-- 1 root root  25751 Apr  2 20:45 training_examples3.tfr
-rw-r--r-- 1 root root  25757 Apr  2 20:45 training_examples4.tfr
-rw-r--r-- 1 root root  25753 Apr  2 20:45 training_examples5.tfr
-rw-r--r-- 1 root root  25743 Apr  2 20:45 training_examples6.tfr
-rw-r--r-- 1 root root  25761 Apr  2 20:45 trai

In [8]:
feature_columns = [
    tf.feature_column.numeric_column('OverallQual', shape=(1,), dtype=tf.float32),
    tf.feature_column.numeric_column('OverallCond', shape=(1,), dtype=tf.float32),
    tf.feature_column.numeric_column('YearBuilt', shape=(1,), dtype=tf.float32),
    tf.feature_column.numeric_column('FullBath', shape=(1,), dtype=tf.float32),
    tf.feature_column.numeric_column('BedroomAbvGr', shape=(1,), dtype=tf.float32),
    tf.feature_column.numeric_column('HalfBath', shape=(1,), dtype=tf.float32),
    tf.feature_column.numeric_column('LotArea', shape=(1,), dtype=tf.float32),
    tf.feature_column.numeric_column('GrLivArea', shape=(1,), dtype=tf.float32),
    tf.feature_column.embedding_column(
        tf.feature_column.categorical_column_with_hash_bucket(
        'Neighborhood', hash_bucket_size=1000),
        dimension=1)
]
print(feature_columns)
label_column = tf.feature_column.numeric_column('SalePrice', shape=(1,), dtype=tf.float32)

features_spec = tf.feature_column.make_parse_example_spec(
    feature_columns + [label_column]
)

[NumericColumn(key='OverallQual', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='OverallCond', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='YearBuilt', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='FullBath', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='BedroomAbvGr', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='HalfBath', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='LotArea', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='GrLivArea', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), EmbeddingColumn(categorical_column=HashedCategoricalColumn(key='Neighborhood', hash_bucket_size=1000, dtype=tf.string), dimension=1, combiner='mean', initializer=<tensorflow.python

In [0]:
def input_fn(file_list):
  dataset = tf.contrib.data.make_batched_features_dataset(
      file_pattern=file_list,
      batch_size=100,
      features=features_spec,
      num_epochs=100,
      shuffle=True,
      shuffle_buffer_size=10
  )
  it = dataset.make_one_shot_iterator()
  features = it.get_next()
  labels = features.pop('SalePrice')
  return features, labels


In [10]:
!rm -r model_dir

rm: cannot remove 'model_dir': No such file or directory


In [11]:
model_dir="model_dir"

regressor = tf.estimator.DNNRegressor(
    hidden_units=[3],
    feature_columns=feature_columns,
    model_dir=model_dir
)

def get_eval_metrics(labels, predictions):
    print("hi")
    print(tf.metrics.root_mean_squared_error(
        labels=labels,
        predictions=predictions['predictions']
    ))
    return {
      'rmse': tf.metrics.root_mean_squared_error(
          labels=labels,
          predictions=predictions['predictions']
      )
  }

regressor = tf.contrib.estimator.add_metrics(regressor, get_eval_metrics)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f3b02be9190>, '_model_dir': 'model_dir', '_protocol': None, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_device_fn': None, '_experimental_distribute': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_evaluation_master': '', '_eval_distribute': None, '_train_distribute': None, '_master': ''}

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/te

In [0]:
train_spec = tf.estimator.TrainSpec(
    input_fn=lambda: input_fn([
        'training_examples1.tfr',
        'training_examples2.tfr',
        'training_examples3.tfr',
        'training_examples4.tfr',
        'training_examples5.tfr',
        'training_examples16tfr',
        'training_examples7.tfr',
        'training_examples8.tfr',
        'training_examples9.tfr',
        'training_examples10.tfr']), #,'training_examples2.tfr','training_examples3.tfr','training_examples4.tfr','training_examples5.tfr','training_examples6.tfr','training_examples7.tfr','training_examples8.tfr','training_examples9.tfr']),
    max_steps=1000000
)

# For evaluation, don't shuffle, and read evaluation data just once.
def eval_input_fn(file_list):
  dataset = tf.contrib.data.make_batched_features_dataset(
      file_pattern=file_list,
      batch_size=100,
      features=features_spec,
      num_epochs=1,
      shuffle=False
      #shuffle_buffer_size=10
  )
  it = dataset.make_one_shot_iterator()
  features = it.get_next()
  labels = features.pop('SalePrice')
  return features, labels

eval_spec = tf.estimator.EvalSpec(
    input_fn=lambda: eval_input_fn([
        'training_examples0.tfr', 
        'training_examples11.tfr',
        'training_examples12.tfr',
        'training_examples13.tfr'])
)


In [13]:
counter = 0

while counter < 50:
  tf.estimator.train_and_evaluate(regressor, train_spec, eval_spec)
  counter = counter + 1
  print(counter)

INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps None or save_checkpoints_secs 600.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use `tf.data.experimental.make_batched_features_dataset(...)`.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:ten

In [14]:
variable_names = regressor.get_variable_names()
print ("variable names: ")
for var_name in variable_names:
  values = regressor.get_variable_value(var_name)
  print (var_name, values)


def get_test_features(row):
  
  OverallQual = row['OverallQual']
  OverallCond = row['OverallCond']
  YearBuilt = row['YearBuilt']
  FullBath = row['FullBath']
  BedroomAbvGr = row['BedroomAbvGr']
  HalfBath = row['HalfBath']
  LotArea = row['LotArea']
  GrLivArea = row['GrLivArea']
  Neighborhood = row['Neighborhood']

  features = {
      "OverallQual": tf.train.Feature(float_list = tf.train.FloatList(value=[OverallQual])),
      "OverallCond": tf.train.Feature(float_list = tf.train.FloatList(value=[OverallCond])),
      "YearBuilt": tf.train.Feature(float_list = tf.train.FloatList(value=[(YearBuilt - 1990) / 10])),
      "FullBath": tf.train.Feature(float_list = tf.train.FloatList(value=[FullBath])),
      "BedroomAbvGr": tf.train.Feature(float_list = tf.train.FloatList(value=[BedroomAbvGr])),
      "HalfBath": tf.train.Feature(float_list = tf.train.FloatList(value=[HalfBath])),
      "LotArea": tf.train.Feature(float_list = tf.train.FloatList(value=[LotArea / 1000.0])),
      "GrLivArea": tf.train.Feature(float_list = tf.train.FloatList(value=[GrLivArea / 1000.0])),
      "Neighborhood": tf.train.Feature(bytes_list = tf.train.BytesList(value=[Neighborhood.encode("utf8")])),
  }
  return features

variable names: 
('dnn/hiddenlayer_0/bias', array([ 1.1524469 ,  1.8664651 , -0.10199566], dtype=float32))
('dnn/hiddenlayer_0/bias/t_0/Adagrad', array([3377955.8 , 5780287.5 ,  104906.55], dtype=float32))
('dnn/hiddenlayer_0/kernel', array([[ 0.30151784, -0.13979737,  0.35539168],
       [-0.11868042,  0.04819114,  0.4158542 ],
       [ 0.36011028,  0.04191711, -0.34054723],
       [-0.37814206,  0.15931936,  0.09083965],
       [ 0.03599561, -0.01354694, -0.3026094 ],
       [-1.2927337 , -1.7263163 , -0.00387851],
       [-0.02151431,  0.03800007,  0.08164231],
       [ 0.5867987 , -0.19845445, -0.26211935],
       [ 0.0806139 , -0.01353709,  0.43100476]], dtype=float32))
('dnn/hiddenlayer_0/kernel/t_0/Adagrad', array([[2.2246758e+07, 3.4129856e+07, 7.1926381e+05],
       [6.9567270e+06, 1.1920682e+07, 3.1865544e+05],
       [7.2510560e+06, 1.3944212e+07, 2.3400156e+05],
       [5.8479031e+05, 1.3431202e+06, 4.1687578e+04],
       [5.6760819e+08, 1.3618052e+09, 5.0575995e+06],
     

In [15]:
import itertools
import math

reader = pd.read_csv('test.csv', iterator = True)
testdf = reader.get_chunk(1459)


testdf['YearBuilt'] = (testdf['YearBuilt'] - 1990) / 10.0
testdf['LotArea'] = testdf['LotArea'] / 1000.
testdf['GrLivArea'] = testdf['GrLivArea'] / 1000.

for index, row in testdf.iterrows():
  OverallQual = row['OverallQual']
  OverallCond = row['OverallCond']
  YearBuilt = row['YearBuilt']
  FullBath = row['FullBath']
  BedroomAbvGr = row['BedroomAbvGr']
  HalfBath = row['HalfBath']
  LotArea = row['LotArea']
  GrLivArea = row['GrLivArea']
  Neighborhood = row['Neighborhood']
  if math.isnan(OverallQual):
    row['OverallQual'] = 5
  if math.isnan(OverallCond):
    row['OverallCond'] = 5
  if math.isnan(YearBuilt):
    row['YearBuilt'] = 0
  if math.isnan(FullBath):
    row['FullBath'] = 2
  if math.isnan(BedroomAbvGr):
    row['BedroomAbvGr'] = 2
    
def eval_input_fn1():
  eval_dataset1 = tf.data.Dataset.from_tensor_slices((dict(reader)))
  #eval_dataset1 = eval_dataset1.shuffle(batch_size).repeat().batch(batch_size)
  eval_dataset1 = eval_dataset1.batch(len(reader))
  return eval_dataset1

def test_input_fn(df):
  x = {
      'OverallQual':tf.constant(
      df['OverallQual'].values,
          shape = [df['OverallQual'].size,1],
          dtype=tf.float32),
      'OverallCond':tf.constant(
      df['OverallCond'].values,
          shape = [df['OverallCond'].size,1],
          dtype=tf.float32),
      'YearBuilt':tf.constant(
      df['YearBuilt'].values,
          shape = [df['YearBuilt'].size,1],
          dtype=tf.float32),
      'FullBath':tf.constant(
      df['FullBath'].values,
          shape = [df['FullBath'].size,1],
          dtype=tf.float32),
      'BedroomAbvGr':tf.constant(
      df['BedroomAbvGr'].values,
          shape = [df['BedroomAbvGr'].size,1],
          dtype=tf.float32),
      'HalfBath':tf.constant(
      df['HalfBath'].values,
          shape = [df['HalfBath'].size,1],
          dtype=tf.float32),
      'LotArea':tf.constant(
      df['LotArea'].values,
          shape = [df['LotArea'].size,1],
          dtype=tf.float32),
      'GrLivArea':tf.constant(
      df['GrLivArea'].values,
          shape = [df['GrLivArea'].size,1],
          dtype=tf.float32),
      'Neighborhood':tf.constant(
      df['Neighborhood'].values,
          shape = [df['Neighborhood'].size,1],
          dtype=tf.string)
  }
  return x

def test_fn_predictions():
  return test_input_fn(testdf)

# For int features:
# 'name': tf.train.Feature(int64_list = tf.train.Int64List(value = ...))
# For float features:
# 'name': tf.train.Feature(float_list = tf.train.FloatList(value = ...))

predictions_generator = regressor.predict(
    input_fn=test_fn_predictions,
    yield_single_examples=True)

iter = itertools.islice(predictions_generator, 1459)

predictions = [y['predictions'][0] for y in iter]

print len(predictions), "predictions:"
print predictions

predictions = [math.exp(i) for i in predictions]
submission = [["Id", "SalePrice"]]
for i in range(1459):
  submission.append([testdf['Id'][i], predictions[i]])
print submission

#   # Print the prediction results.
# print("\nPrediction results:")
# print predict_results
# print next(predict_results)
# for i, prediction in enumerate(predict_results):
#   print prediction

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_dir/model.ckpt-45000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
1459 predictions:
[11.747725, 11.959175, 12.022488, 12.145509, 12.328175, 12.09494, 12.029954, 12.043644, 12.091951, 11.6252985, 12.142363, 11.533301, 11.45957, 11.949924, 11.857394, 12.861649, 12.507865, 12.631435, 12.591429, 12.967062, 12.7202215, 12.43904, 12.056832, 12.054791, 12.140184, 12.246796, 12.695419, 12.3906145, 12.187972, 12.171816, 12.217684, 11.535203, 12.074171, 12.621154, 12.5692215, 12.247472, 12.30454, 12.1411085, 12.138989, 12.06459, 12.141329, 12.094309, 12.380604, 12.43028, 12.232517, 12.11828, 12.385139, 12.170408, 12.064542, 11.800476, 11.797396, 11.947728, 12.069861, 11.957531, 12.265425, 12.012709, 12.03343, 11.873654, 12.193532, 